In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt


In [7]:
plt.switch_backend('QtAgg')
plt.ion() #Makes plot interactive

# get raw data from .edf file
raw = mne.io.read_raw_edf("eotestbrite34.edf",encoding='latin1')
# raw = mne.io.read_raw_nirx("eotestbrite34.edf", verbose=True)

# get annotation (events) from edf file (edf+)
events_from_annot, event_dict = mne.events_from_annotations(raw)
print(event_dict)
print(events_from_annot[:5])
event_id = {'eventButton':1, 'Фоновая запись (testUser)':2}
# show events
fig = mne.viz.plot_events(events_from_annot, event_id=event_id,
                          sfreq=raw.info['sfreq'],
                          first_samp=events_from_annot[0, 0])

# keep the color-code for further plotting
stage_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# plot some data
# scalings were chosen manually to allow for simultaneous visualization of
# different channel types in this specific dataset
raw.plot(start=0, duration=60,
               scalings=dict(eeg=1e-4, resp=1e3, eog=1e-4, emg=1e-7,
                             misc=1e-1), lowpass=40)

Extracting EDF parameters from c:\opencv\eotestbrite34.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
{}
[]


C:\Users\artyo\AppData\Local\Temp\ipykernel_5500\3280465893.py:5: RuntimeWarning: Channel names are not unique, found duplicates for: {'Rx8 - Tx10 HHb', 'Rx8 - Tx9 HHb', 'Rx4 - Tx8 HHb', 'Rx7 - Tx7 HHb', 'Rx6 - Tx8 HHb', 'Rx8 - Tx9 O2Hb', 'Rx4 - Tx3 HHb', 'Rx1 - Tx1 HHb', 'Rx2 - Tx4 HHb', 'Rx4 - Tx8 O2Hb', 'Rx2 - Tx3 HHb', 'Rx4 - Tx3 O2Hb', 'Rx3 - Tx2 HHb', 'Rx1 - Tx1 O2Hb', 'Rx6 - Tx4 HHb', 'Rx6 - Tx9 O2Hb', 'Rx1 - Tx2 O2Hb', 'Rx3 - Tx5 O2Hb', 'Rx1 - Tx3 O2Hb', 'Rx2 - Tx4 O2Hb', 'Rx5 - Tx5 O2Hb', 'Rx1 - Tx2 HHb', 'Rx7 - Tx10 O2Hb', 'Rx1 - Tx3 HHb', 'Rx7 - Tx8 O2Hb', 'Rx4 - Tx5 O2Hb', 'Rx4 - Tx4 O2Hb', 'Rx7 - Tx7 O2Hb', 'Rx2 - Tx1 O2Hb', 'Rx5 - Tx7 HHb', 'Rx2 - Tx1 HHb', 'Rx3 - Tx2 O2Hb', 'Rx3 - Tx6 HHb', 'Rx4 - Tx5 HHb', 'Rx3 - Tx3 HHb', 'Rx5 - Tx6 O2Hb', 'Rx8 - Tx10 O2Hb', 'Rx8 - Tx8 HHb', 'Rx3 - Tx5 HHb', 'Rx7 - Tx8 HHb', 'Rx3 - Tx6 O2Hb', 'Rx7 - Tx5 HHb', 'Rx3 - Tx3 O2Hb', 'Rx6 - Tx9 HHb', 'Rx6 - Tx4 O2Hb', 'Rx7 - Tx10 HHb', 'Rx5 - Tx6 HHb', 'Rx5 - Tx7 O2Hb', 'Rx7 - Tx5 O2Hb', 'Rx8

IndexError: index 0 is out of bounds for axis 0 with size 0

In [3]:
spectrum = raw.compute_psd()
spectrum.plot(average=True)

Effective window size : 0.512 (s)


<MNELineFigure size 1250x437.5 with 1 Axes>

In [4]:
# midline = ['EEG F4-AA','EEG O1-AA']
# spectrum.plot(picks=midline)

In [5]:

# Extract 15 s epochs relative to events, baseline correct, linear detrend, and reject 
# epochs where eeg amplitude is > 400
epochs = mne.Epochs(raw, events_from_annot, event_id=event_id, tmin=0,
                    tmax=15, baseline=(0, 0), reject=dict(eeg=0.000400), detrend=1)  

# Drop bad trials
epochs.drop_bad()

Not setting metadata
18 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 18 events and 7501 original time points ...
    Rejecting  epoch based on EEG : ['EEG O2-AA', 'EEG CZ-AA']
    Rejecting  epoch based on EEG : ['EEG F3-AA']
    Rejecting  epoch based on EEG : ['EEG F3-AA']
4 bad epochs dropped


Number of events,14
Events,eventButton: 14Фоновая запись (testUser): 0
Time range,0.000 – 15.000 sec
Baseline,0.000 – 0.000 sec


In [6]:
# Average erpochs to form ERPs
attend6 = epochs['eventButton'].average()

# Plot ERPs
evokeds = dict(attend6=list(epochs['eventButton'].iter_evoked()))
mne.viz.plot_compare_evokeds(evokeds, combine='mean')

combining channels using "mean"
Channels marked as bad:
none


[<Figure size 1000x750 with 1 Axes>]